In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.718061,"""a""",0.121634,0.763113,0.732562,0.377112,0.375955,-0.851582
0.841471,1,"""a""",0,0.699118,"""a""",0.962587,0.096575,0.673011,0.738888,0.100335,-0.83615
0.909297,2,"""a""",1,0.805626,"""a""",0.813999,0.460178,0.514103,0.51099,0.785711,-0.510919
0.14112,3,"""a""",1,0.00557,"""a""",0.977954,0.687947,0.828845,0.067208,0.464131,-0.890152
-0.756802,4,"""a""",0,0.668774,"""a""",0.743661,0.900544,0.401849,0.629593,0.745616,-0.221055


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.377112,-0.375955
-0.841471,-0.738888,-0.100335
-0.909297,-0.51099,-0.785711
-0.14112,-0.067208,-0.464131
0.756802,-0.25248,-0.369661


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

lstsq_coeffs
list[f64]
"[-0.504704, -0.337103]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.219255,0.001702,128.82251,0.0,0.215919,0.222592
"""exp(x2)""",0.175347,0.000682,257.08593,0.0,0.17401,0.176684
"""sin(x3)""",-1.745287,0.001352,-1290.855454,0.0,-1.747937,-1.742636
"""__bias__""",-0.108876,0.001515,-71.843245,0.0,-0.111847,-0.105905


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

lstsq_coeffs
list[f64]
"[-0.504704, -0.337103]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.499809, -0.347422]"
"""a""","[-0.499809, -0.347422]"
"""a""","[-0.499809, -0.347422]"
"""a""","[-0.499809, -0.347422]"
"""a""","[-0.499809, -0.347422]"
…,…
"""b""","[-0.509563, -0.32691]"
"""b""","[-0.509563, -0.32691]"
"""b""","[-0.509563, -0.32691]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.121634,0.763113,-0.851582,-0.318637,-0.532946
0.962587,0.096575,-0.83615,-0.518377,-0.317772
0.813999,0.460178,-0.510919,-0.565955,0.055036
0.977954,0.687947,-0.890152,-0.725486,-0.164666
0.743661,0.900544,-0.221055,-0.678904,0.457849


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.499809, -0.347422]"
"""b""","[-0.509563, -0.32691]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""b""","[-0.336282, -0.155478]"
"""a""","[-0.336463, -0.167543]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.query_lstsq(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.543494
"""a""",-0.534178


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.query_rolling_lstsq(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.851582,0.121634,0.763113,null,null
-0.83615,0.962587,0.096575,null,null
-0.510919,0.813999,0.460178,null,null
-0.890152,0.977954,0.687947,null,null
-0.221055,0.743661,0.900544,"[-0.536204, -0.362532]",-0.72523
…,…,…,…,…
-0.019252,0.086137,0.78473,"[-0.859638, -0.002479]",-0.075992
-0.564748,0.577775,0.371494,"[-1.009086, 0.089103]",-0.549923
-0.661185,0.895924,0.83152,"[-0.957789, 0.219304]",-0.67575


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[29.003027, 28.800535, 28.766516]"


In [18]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.989289,"[-0.344595, 0.938751]"
28.798623,"[0.938751, 0.344595]"


In [19]:
# PC1
df.select(
    pds.query_principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.081069
-0.464474
0.257457
0.108498
0.178948


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.333429,1.002914,0.520892,0.496904,0.508616,0.524247,0.499725
"""a""",0.332551,0.994944,0.48794,0.505324,0.49648,0.489441,0.502887


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""church"""
"""going"""
"""to"""
"""world"""
"""hello"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""world"""
"""hello"""
"""church"""
""""""
"""go"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [30]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [31]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.012492
0.595561
0.185681


In [32]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,2.978394,null
null,0.970464,null
-0.012492,-0.072767,0.292304
0.595561,0.286816,1.661764
0.185681,0.591648,1.802518


In [33]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""8kwTY""",null
null,"""kMp""",null
-0.012492,"""Rc37s""","""W1"""
0.595561,"""5""","""Xj5GN"""
0.185681,"""v7fD""","""1buvo"""


In [34]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.012492,"""kecis"""
0.595561,"""u9Vh3"""
0.185681,"""0adxI"""


In [35]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.261042,null,-0.261344
null,-0.527867,null,-0.527969
-0.012492,-0.631845,2.562054,-0.631728
0.595561,0.673208,1.030199,0.672906
0.185681,0.572215,2.065594,0.572634


In [36]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.106948,0.427388,5
null,0.095822,-0.798821,9
-0.012492,0.388392,-0.185242,2
0.595561,0.042763,0.802807,4
0.185681,0.321888,0.277879,2


In [37]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.57873,0.562857,7.284288,0.026196


In [38]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.099163,0.187826,3,1
1,0.212648,0.759222,4,2
2,0.698327,0.597246,1,6
0,0.445486,0.3729,4,4
1,0.649741,0.974461,4,9


In [39]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.03261,0.973986}","{27.618037,0.840533}","{0.849663,0.493529}"


In [40]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.query_chi2("category_1", "category_2").alias("chi2-test"),
        pds.query_f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {1.320059,0.186906}  ┆ {36.690713,0.436664} ┆ {0.2116,0.932081}   │
│ 1         ┆ {-0.123313,0.901867} ┆ {29.961059,0.750506} ┆ {1.363618,0.244265} │
│ 2         ┆ {-1.237752,0.215895} ┆ {26.494933,0.876345} ┆ {0.36564,0.833211}  │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [41]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
567,0.1134
558,0.1116
567,0.1134
564,0.1128
544,0.1088
588,0.1176
529,0.1058
548,0.1096
535,0.107


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [42]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [43]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.481702,0.566173,0.69857,0.458351,7.900392,14
1,0.176238,0.691061,0.38883,0.873326,9.814184,22
2,0.201282,0.876521,0.566095,0.046258,3.17553,20
3,0.859807,0.967636,0.785815,0.804368,9.882445,11
4,0.064274,0.618677,0.463781,0.232584,8.62535,20


In [44]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.481702,0.566173,0.69857,0.458351,7.900392,233
1,0.176238,0.691061,0.38883,0.873326,9.814184,968
2,0.201282,0.876521,0.566095,0.046258,3.17553,1
3,0.859807,0.967636,0.785815,0.804368,9.882445,421
4,0.064274,0.618677,0.463781,0.232584,8.62535,32


In [45]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.481702,0.566173,0.69857,0.458351,7.900392,"[0, 1220, … 61]"
1,0.176238,0.691061,0.38883,0.873326,9.814184,"[1, 1919, … 1438]"
2,0.201282,0.876521,0.566095,0.046258,3.17553,"[2, 1530, … 1511]"
3,0.859807,0.967636,0.785815,0.804368,9.882445,"[3, 1734, … 838]"
4,0.064274,0.618677,0.463781,0.232584,8.62535,"[4, 348, … 307]"


In [46]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1220, … 687]  ┆ 4                  │
│ 1   ┆ [1, 1919, … 254]  ┆ 14                 │
│ 2   ┆ [2, 1530, … 1457] ┆ 8                  │
│ 3   ┆ [3, 1734, … 820]  ┆ 5                  │
│ 4   ┆ [4, 348, … 1888]  ┆ 13                 │
└─────┴───────────────────┴────────────────────┘


In [47]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.481702,0.566173,0.69857,0.458351,7.900392,"[0, 1220, … 61]","[0.0, 0.033199, … 0.060468]"
1,0.176238,0.691061,0.38883,0.873326,9.814184,"[1, 1919, … 1438]","[0.0, 0.022711, … 0.06655]"
2,0.201282,0.876521,0.566095,0.046258,3.17553,"[2, 1530, … 1511]","[0.0, 0.067087, … 0.076166]"
3,0.859807,0.967636,0.785815,0.804368,9.882445,"[3, 1734, … 838]","[0.0, 0.023593, … 0.07114]"
4,0.064274,0.618677,0.463781,0.232584,8.62535,"[4, 348, … 307]","[0.0, 0.062754, … 0.076836]"


In [48]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.481702,0.566173,0.69857,0.458351,7.900392
1,0.176238,0.691061,0.38883,0.873326,9.814184
5,0.712239,0.209799,0.535259,0.505202,7.753107
7,0.854959,0.645341,0.465969,0.924045,8.752052
8,0.797774,0.567559,0.411024,0.193817,9.037423


In [49]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.481702,0.566173,0.69857,0.458351,7.900392
40,0.41957,0.46582,0.289675,0.797387,1.012019
61,0.492211,0.532409,0.747621,0.607236,6.230372
66,0.445782,0.557399,0.566395,0.87715,5.277144
138,0.512836,0.538711,0.918922,0.563493,1.969114


In [50]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
0,0.481702,0.566173,0.69857,0.458351,7.900392
61,0.492211,0.532409,0.747621,0.607236,6.230372
180,0.522403,0.494358,0.795411,0.971752,7.740899
377,0.520169,0.449136,0.262714,0.088863,8.283747
480,0.472071,0.455911,0.734145,0.448297,6.728847


In [51]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 911, 1130]",3
1,"[1, 1919]",2
2,[2],1
3,"[3, 548, 1734]",3
4,"[4, 981, … 343]",5


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [52]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [53]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [54]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [55]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [56]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"


# Using PDS Expressions On Series / NumPy arrays

In [57]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
1.0,0.333752,1,9
1.0,0.766684,0,7
0.0,0.977014,2,2
1.0,0.768644,2,2
0.0,0.173281,0,2


In [58]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "query_jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [59]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.497343,0.497721,0.497532,0.49837,0.497449


In [60]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "query_psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.202925,0.2,0.2,0.0
0.401657,0.2,0.188,0.000743
0.605492,0.2,0.223,0.002504
0.812192,0.2,0.206,0.000177
inf,0.2,0.183,0.00151


In [61]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
12.610733


In [62]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.128507


# Examples using Ball Tree Features

Not currently available.

In [63]:
# size = 1_000
# df = pl.DataFrame({
#     "f": np.sin(list(range(size)))
#     , "time_idx": range(size)
#     , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
#     , "actual": np.round(np.random.random(size=size)).astype(np.int32)
#     , "predicted": np.random.random(size=size)
#     , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
# }).with_columns(
#     pds.random(0., 1.).alias("x1")
#     , pds.random(0., 1.).alias("x2")
#     , pds.random(0., 1.).alias("x3")
#     , pds.random(0., 1.).alias("a")
#     , pds.random(0., 1.).alias("b")
# ).with_columns(
#     y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
# )
# df = df.with_columns(pl.int_range(0, size).cast(dtype=pl.UInt32).alias("index"))
# df.head()

In [64]:
# # Pointwise Nearest Neighbors
# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         parallel=True).alias("ball_tree_knn_ptwise")
# )

In [65]:
# # Pointwise Nearest Neighbors with distances
# # we get a struct with an ids column and a dist column

# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         return_dist=True,
#         parallel=True).alias("ball_tree_knn_ptwise")
# ).unnest("ball_tree_knn_ptwise")

In [66]:
# # Frequency Count
# df.with_columns(
#     pds.query_bt_knn_radius_freq_cnt(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =5, 
#         distance_metric="haversine", 
#         parallel=True
#         ).alias("ball_tree_knn_radius_freq_cnt")
# )

In [67]:
# # Average distance of k nearest neighbors
# df.with_columns(
#     pds.query_bt_knn_avg(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         pl.col("x1"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =1,
#         distance_metric="euclidean", 
#         parallel=True).alias("btree_knn_avg")
# )

In [68]:
# # Count neighbors within radius
# df.with_columns(
#     pds.query_bt_nb_cnt(
#         pl.col("x2"),
#         pl.col("x3"),
#         index=pl.col("index"),
#         r=18.0, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_radius")
# )

In [69]:
# # returns true if the row is within k nearest neighbors of the given point
# df.with_columns(
#     pds.bt_within_dist_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=[123,78.99],
#         r=999999, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_dist_from")
# )

In [70]:
# # check if a point is included in the k nearest neighbors of the given point
# # Note: This is an exact match so your point could be quite close but now show up
# # We can pass in an EPSILON to account for this. THis is defaulted to the EPSILON of the data type in Rust

# # pick a random point
# point = df.select(pl.col("x2"), pl.col("x3"))
# rr = list(point.row(2))
# df.with_columns(
#     pds.is_bt_knn_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=rr, 
#         k=56, 
#         distance_metric="haversine",
#         parallel=True, epsilon=0.5).alias("btree_nn_within")
# )